<a href="https://colab.research.google.com/github/ChauHPham/AIorHumanText/blob/main/quiz_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Text Detector Quiz - Google Colab

A fun quiz game where you guess whether text samples are AI-generated or human-written!

This notebook can:
1. Run the quiz locally with ngrok (temporary URL)
2. Deploy to Hugging Face Spaces (permanent URL) 🚀


# AI Text Detector Quiz - Google Colab

A fun quiz game where you guess whether text samples are AI-generated or human-written!

This notebook runs the quiz app in Google Colab, avoiding MPS errors on M2 Macs.


## Step 1: Install Dependencies


In [1]:
%pip install flask flask-cors pandas scikit-learn torch transformers pyyaml pyngrok -q


## Step 2: Upload Dataset

Upload your CSV files (`ai_vs_human_text.csv` and/or `dataset.csv`) using the file uploader below.


In [2]:
from google.colab import files
import os

# Create data directory
os.makedirs('data', exist_ok=True)

# Upload files
uploaded = files.upload()

# Move CSV files to data directory
for filename in uploaded.keys():
    if filename.endswith('.csv'):
        os.rename(filename, f'data/{filename}')
        print(f'✓ Uploaded {filename} to data/')


Saving dataset.csv to dataset.csv
✓ Uploaded dataset.csv to data/


## Step 3: Create Quiz Files

The following cells will create all necessary files for the quiz app.


In [3]:
# Create src directory and quiz dataset loader
os.makedirs('src', exist_ok=True)

# Write quiz_dataset_loader.py
quiz_loader_code = '''"""
Quiz dataset loader - handles loading quiz text samples from CSV files
"""
import os
import random
import pandas as pd
import logging

logger = logging.getLogger(__name__)

class QuizDatasetLoader:
    """
    Loads quiz text samples from CSV files
    """
    def __init__(self, data_dir='data', csv_files=None):
        self.data_dir = data_dir
        self.csv_files = csv_files or ['ai_vs_human_text.csv', 'dataset.csv']
        self.samples = []
        self.class_names = ['Human-written', 'AI-generated']
        self.load_samples()

    def load_samples(self):
        """Load samples from CSV files"""
        all_samples = []

        for csv_file in self.csv_files:
            csv_path = os.path.join(self.data_dir, csv_file)
            if not os.path.exists(csv_path):
                logger.warning(f"CSV file not found: {csv_path}")
                continue

            try:
                df = pd.read_csv(csv_path)

                # Normalize column names
                text_col = None
                label_col = None

                # Find text column
                for col in ['text', 'content', 'body', 'essay']:
                    if col in df.columns:
                        text_col = col
                        break

                # Find label column
                for col in ['label', 'target', 'class', 'is_ai']:
                    if col in df.columns:
                        label_col = col
                        break

                if text_col is None or label_col is None:
                    logger.warning(f"Could not find text or label column in {csv_file}")
                    continue

                # Normalize labels
                def normalize_label(label):
                    if pd.isna(label):
                        return None
                    label_str = str(label).strip().lower()
                    if label_str in ['ai-generated', 'ai', 'machine', 'generated', 'gpt', 'llm', 'chatgpt', '1']:
                        return 1  # AI-generated
                    elif label_str in ['human-written', 'human', 'person', 'authored', 'real', '0']:
                        return 0  # Human-written
                    return None

                # Process rows
                for idx, row in df.iterrows():
                    text = str(row[text_col]).strip()
                    label = normalize_label(row[label_col])

                    if text and text != 'nan' and label is not None:
                        all_samples.append({
                            'id': len(all_samples),
                            'text': text,
                            'label': label,
                            'label_name': self.class_names[label]
                        })

                logger.info(f"Loaded {len(df)} rows from {csv_file}")

            except Exception as e:
                logger.error(f"Error loading {csv_file}: {e}")
                import traceback
                logger.error(traceback.format_exc())

        if all_samples:
            self.samples = all_samples
            logger.info(f"Loaded {len(self.samples)} quiz text samples total")
        else:
            logger.warning("No quiz samples available. Quiz will not work.")
            logger.warning("Please ensure CSV files exist in the data/ directory")
            self.samples = []

    def get_random_sample(self):
        """Get a random sample"""
        if not self.samples:
            return None, None
        idx = random.randint(0, len(self.samples) - 1)
        return idx, self.get_sample(idx)

    def get_sample(self, idx):
        """Get sample by index"""
        if idx < 0 or idx >= len(self.samples):
            return None
        return self.samples[idx]

    def __len__(self):
        return len(self.samples)
'''

with open('src/__init__.py', 'w') as f:
    f.write('# Quiz module\n')

with open('src/quiz_dataset_loader.py', 'w') as f:
    f.write(quiz_loader_code)

print('✓ Created src/quiz_dataset_loader.py')


✓ Created src/quiz_dataset_loader.py


In [5]:
# Create app.py (Colab-compatible, no MPS code)\n
app_code_colab = '''import os
import sys
import random
import logging
from flask import Flask, request, jsonify, render_template
from flask_cors import CORS

import torch
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Import the detector model and quiz loader
from ai_text_detector.models import DetectorModel
from src.quiz_dataset_loader import QuizDatasetLoader

app = Flask(__name__)
CORS(app)

# Global detector instance and quiz dataset
detector = None
quiz_loader = None

@app.route('/')
def index():
    """Serve the main page"""
    return render_template('index.html')

@app.route('/health')
def health():
    """Health check endpoint"""
    quiz_status = 'loaded' if (quiz_loader and len(quiz_loader) > 0) else 'not loaded'

    return jsonify({
        'status': 'healthy',
        'model_loaded': detector is not None,
        'device': str(next(detector.model.parameters()).device) if detector else 'unknown',
        'quiz_loader_status': quiz_status,
        'quiz_loader_count': len(quiz_loader) if quiz_loader else 0
    })

@app.route('/quiz/text', methods=['GET'])
def get_quiz_text():
    """Get a random text sample from the quiz dataset"""
    try:
        if quiz_loader and len(quiz_loader) > 0:
            idx, sample_data = quiz_loader.get_random_sample()
            if sample_data:
                return jsonify({
                    'text_id': idx,
                    'text': sample_data['text'],
                    'true_label': sample_data['label_name']
                })
            else:
                logger.error(f"Failed to get sample data for index {idx}")
                return jsonify({'error': 'Failed to load text data'}), 500
        else:
            error_msg = 'Quiz dataset not available. '
            if not quiz_loader:
                error_msg += 'Quiz loader not initialized. '
            elif len(quiz_loader) == 0:
                error_msg += 'Quiz loader has 0 samples. '
            error_msg += 'Please check data/ directory for CSV files.'

            logger.error(error_msg)
            return jsonify({'error': error_msg}), 500
    except Exception as e:
        logger.error(f"Error getting quiz text: {e}")
        import traceback
        logger.error(traceback.format_exc())
        return jsonify({'error': str(e)}), 500

@app.route('/quiz/text/<int:text_id>', methods=['GET'])
def get_quiz_text_by_id(text_id):
    """Get a specific text sample by ID"""
    try:
        if quiz_loader and text_id < len(quiz_loader):
            sample = quiz_loader.get_sample(text_id)
            if sample:
                return jsonify({
                    'text_id': text_id,
                    'text': sample['text'],
                    'true_label': sample['label_name']
                })
            else:
                return jsonify({'error': f'Text sample {text_id} not found'}), 404

        return jsonify({'error': f'Invalid text ID: {text_id}'}), 404
    except Exception as e:
        logger.error(f"Error getting quiz text {text_id}: {e}")
        import traceback
        logger.error(traceback.format_exc())
        return jsonify({'error': str(e)}), 500

@app.route('/quiz/check', methods=['POST'])
def check_quiz_answer():
    """Check user's answer and return result with model prediction"""
    try:
        data = request.json
        text_id = data.get('text_id')
        user_answer = data.get('answer')  # 'AI-generated' or 'Human-written'

        if text_id is None or user_answer is None:
            return jsonify({'error': 'Missing text_id or answer'}), 400

        # Get text and label from quiz_loader
        if quiz_loader is None or text_id >= len(quiz_loader):
            return jsonify({'error': 'Invalid text ID'}), 404

        sample = quiz_loader.get_sample(text_id)
        if sample is None:
            return jsonify({'error': 'Text sample not found'}), 404

        text = sample['text']
        true_label = sample['label_name']

        # Get model prediction
        ai_prob, predicted_label = detector.predict(text, max_length=768, threshold=0.5)

        # Map predicted label to label name
        predicted_label_name = 'AI-generated' if predicted_label == 1 else 'Human-written'

        # Calculate human probability
        human_prob = 1 - ai_prob

        # Check if user is correct
        is_correct = user_answer == true_label

        return jsonify({
            'is_correct': is_correct,
            'user_answer': user_answer,
            'true_label': true_label,
            'model_prediction': predicted_label_name,
            'model_confidence': ai_prob if predicted_label == 1 else human_prob,
            'model_probabilities': {
                'AI-generated': ai_prob,
                'Human-written': human_prob
            }
        })
    except Exception as e:
        logger.error(f"Error checking quiz answer: {e}")
        import traceback
        logger.error(traceback.format_exc())
        return jsonify({'error': str(e)}), 500

def load_detector():
    """Load the AI Text Detector"""
    global detector

    model_path = "models/ai_detector"

    # Check if model directory exists AND has model files
    has_model = False
    if os.path.exists(model_path):
        # Check for required model files
        required_files = ["config.json", "pytorch_model.bin"]
        has_model = all(os.path.exists(os.path.join(model_path, f)) for f in required_files)

    if has_model:
        try:
            logger.info(f"Loading trained model from {model_path}")
            detector = DetectorModel.load(model_path)
        except Exception as e:
            logger.error(f"Failed to load model: {e}")
            logger.info("Using Desklib pre-trained model instead.")
            detector = DetectorModel("desklib/ai-text-detector-v1.01", use_desklib=True)
    else:
        logger.info("No trained model found. Loading Desklib pre-trained AI detector model...")
        try:
            detector = DetectorModel("desklib/ai-text-detector-v1.01", use_desklib=True)
            logger.info("✅ Desklib model loaded successfully!")
        except Exception as e:
            logger.error(f"⚠️  Failed to load Desklib model: {e}")
            logger.info("Falling back to RoBERTa base model (will need training for good results)...")
            import traceback
            traceback.print_exc()
            detector = DetectorModel("roberta-base", use_desklib=False)

    return detector

def load_quiz_dataset(data_dir='data'):
    """Load the quiz dataset from CSV files"""
    global quiz_loader

    logger.info(f"Loading quiz dataset from {data_dir}...")

    try:
        quiz_loader = QuizDatasetLoader(data_dir=data_dir)

        if len(quiz_loader) > 0:
            msg = f"✓ Quiz dataset loaded: {len(quiz_loader)} text samples"
            print(msg)
            logger.info(msg)
            return quiz_loader
        else:
            logger.warning("QuizDatasetLoader created but has 0 samples")
    except Exception as e:
        error_msg = f"Warning: Could not load quiz dataset: {e}"
        print(error_msg)
        logger.error(error_msg, exc_info=True)
        quiz_loader = None

    warning_msg = "⚠️  No quiz dataset available. Quiz will not work."
    print(warning_msg)
    logger.warning(warning_msg)
    print("   Please ensure CSV files exist in the data/ directory")
    return None

# Initialize on import (for gunicorn) - must be after function definitions
def init_app():
    """Initialize the app - called on startup"""
    global detector, quiz_loader

    # Load detector
    try:
        load_detector()
    except Exception as e:
        logger.error(f"Failed to load detector: {e}")

    # Load quiz dataset
    try:
        load_quiz_dataset()
    except Exception as e:
        logger.error(f"Failed to load quiz dataset: {e}")

# Initialize when module is imported (works with gunicorn)
init_app()

if __name__ == '__main__':
    # Load detector on startup (already done by init_app, but keep for direct execution)
    if detector is None:
        load_detector()
    if quiz_loader is None:
        load_quiz_dataset()
    # Get port from environment variable (for deployment) or use default
    port = int(os.environ.get('PORT', 5000))
    app.run(debug=False, host='0.0.0.0', port=port)

'''

with open('app.py', 'w') as f:
    f.write(app_code_colab)
print('✓ Created app.py')

✓ Created app.py


In [7]:
os.makedirs('templates', exist_ok=True)
html_template = '''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AI Text Detector Quiz</title>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }

        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            display: flex;
            align-items: center;
            justify-content: center;
            padding: 20px;
        }

        .container {
            background: white;
            border-radius: 20px;
            box-shadow: 0 20px 40px rgba(0, 0, 0, 0.1);
            padding: 40px;
            max-width: 900px;
            width: 100%;
            text-align: center;
        }

        h1 {
            color: #333;
            margin-bottom: 10px;
            font-size: 2.5em;
            font-weight: 300;
        }

        .subtitle {
            color: #666;
            margin-bottom: 20px;
            font-size: 1.1em;
        }

        .score-board {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 20px;
            border-radius: 15px;
            margin-bottom: 30px;
            display: flex;
            justify-content: space-around;
            align-items: center;
        }

        .score-item {
            text-align: center;
        }

        .score-label {
            font-size: 0.9em;
            opacity: 0.9;
            margin-bottom: 5px;
        }

        .score-value {
            font-size: 2em;
            font-weight: bold;
        }

        .quiz-text-container {
            margin: 30px 0;
            position: relative;
        }

        .quiz-text {
            background: #f8f9ff;
            border: 2px solid #e0e0e0;
            border-radius: 15px;
            padding: 30px;
            max-height: 400px;
            overflow-y: auto;
            text-align: left;
            font-size: 1.1em;
            line-height: 1.8;
            color: #333;
            box-shadow: 0 5px 15px rgba(0, 0, 0, 0.1);
        }

        .text-loading {
            min-height: 200px;
            display: flex;
            align-items: center;
            justify-content: center;
            background: #f5f5f5;
            border-radius: 15px;
        }

        .answer-buttons {
            display: flex;
            gap: 20px;
            justify-content: center;
            margin: 30px 0;
        }

        .btn {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            border: none;
            padding: 20px 50px;
            border-radius: 25px;
            font-size: 1.3em;
            font-weight: bold;
            cursor: pointer;
            transition: all 0.3s ease;
            flex: 1;
            max-width: 250px;
        }

        .btn:hover:not(:disabled) {
            transform: translateY(-3px);
            box-shadow: 0 10px 25px rgba(102, 126, 234, 0.4);
        }

        .btn:active:not(:disabled) {
            transform: translateY(-1px);
        }

        .btn:disabled {
            opacity: 0.6;
            cursor: not-allowed;
        }

        .btn-ai {
            background: linear-gradient(135deg, #e17055 0%, #d63031 100%);
        }

        .btn-human {
            background: linear-gradient(135deg, #00b894 0%, #00a085 100%);
        }

        .btn-next {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            margin-top: 20px;
        }

        .result-panel {
            display: none;
            margin-top: 30px;
            padding: 30px;
            background: #f8f9ff;
            border-radius: 15px;
            border-left: 5px solid #667eea;
            animation: fadeIn 0.5s ease;
        }

        .result-panel.show {
            display: block;
        }

        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(20px); }
            to { opacity: 1; transform: translateY(0); }
        }

        .result-title {
            font-size: 2em;
            font-weight: bold;
            margin-bottom: 20px;
        }

        .result-correct {
            color: #00b894;
        }

        .result-incorrect {
            color: #d63031;
        }

        .result-details {
            margin: 20px 0;
            font-size: 1.1em;
        }

        .result-detail-item {
            margin: 10px 0;
            padding: 10px;
            background: white;
            border-radius: 8px;
        }

        .model-prediction {
            margin-top: 20px;
            padding: 20px;
            background: white;
            border-radius: 10px;
        }

        .model-prediction-title {
            font-weight: bold;
            margin-bottom: 15px;
            font-size: 1.2em;
        }

        .probabilities {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(150px, 1fr));
            gap: 15px;
            margin-top: 15px;
        }

        .prob-item {
            background: #f8f9ff;
            padding: 15px;
            border-radius: 10px;
        }

        .prob-label {
            font-weight: bold;
            margin-bottom: 8px;
        }

        .prob-bar {
            background: #e0e0e0;
            height: 10px;
            border-radius: 5px;
            overflow: hidden;
            margin-bottom: 5px;
        }

        .prob-fill {
            height: 100%;
            background: linear-gradient(90deg, #667eea, #764ba2);
            transition: width 0.5s ease;
        }

        .prob-value {
            font-size: 0.9em;
            color: #666;
        }

        .loading {
            display: none;
            margin: 20px 0;
        }

        .spinner {
            border: 4px solid #f3f3f3;
            border-top: 4px solid #667eea;
            border-radius: 50%;
            width: 40px;
            height: 40px;
            animation: spin 1s linear infinite;
            margin: 0 auto;
        }

        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }

        .error {
            background: #ffe6e6;
            color: #d63031;
            padding: 15px;
            border-radius: 10px;
            margin: 20px 0;
            border-left: 5px solid #d63031;
        }

        .start-screen {
            text-align: center;
        }

        .start-btn {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            border: none;
            padding: 20px 60px;
            border-radius: 25px;
            font-size: 1.5em;
            font-weight: bold;
            cursor: pointer;
            transition: all 0.3s ease;
            margin-top: 30px;
        }

        .start-btn:hover {
            transform: translateY(-3px);
            box-shadow: 0 10px 25px rgba(102, 126, 234, 0.4);
        }

        .quiz-screen {
            display: none;
        }

        .quiz-screen.active {
            display: block;
        }

        .question-selector {
            margin: 30px 0;
        }

        .question-selector-label {
            font-size: 1.1em;
            color: #666;
            margin-bottom: 15px;
            font-weight: 500;
        }

        .question-options {
            display: flex;
            gap: 15px;
            justify-content: center;
            flex-wrap: wrap;
        }

        .question-option {
            background: white;
            border: 3px solid #ddd;
            padding: 15px 30px;
            border-radius: 15px;
            font-size: 1.2em;
            font-weight: bold;
            cursor: pointer;
            transition: all 0.3s ease;
            color: #666;
        }

        .question-option:hover {
            border-color: #667eea;
            color: #667eea;
            transform: translateY(-2px);
        }

        .question-option.selected {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            border-color: #667eea;
            color: white;
        }

        .progress-indicator {
            background: #f8f9ff;
            padding: 15px;
            border-radius: 10px;
            margin-bottom: 20px;
            font-size: 1.1em;
            color: #666;
            font-weight: 500;
        }

        .completion-screen {
            display: none;
            text-align: center;
        }

        .completion-screen.active {
            display: block;
        }

        .completion-title {
            font-size: 2.5em;
            margin-bottom: 20px;
            color: #333;
        }

        .completion-stats {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            border-radius: 15px;
            margin: 30px 0;
        }

        .completion-stat {
            margin: 15px 0;
            font-size: 1.3em;
        }

        .completion-stat-value {
            font-size: 2em;
            font-weight: bold;
            margin-top: 5px;
        }
    </style>
</head>
<body>
    <div class="container">
        <!-- Start Screen -->
        <div class="start-screen" id="startScreen">
            <h1>📝 AI Text Detector Quiz</h1>
            <p class="subtitle">Test your ability to distinguish AI-generated text from human-written text!</p>
            <p class="subtitle" style="margin-top: 20px; color: #999; font-size: 0.95em;">
                You'll be shown text samples one at a time. Guess whether each text is AI-generated or Human-written,
                then see if you're correct and how the AI model predicts it.
            </p>
            <div class="question-selector">
                <div class="question-selector-label">Select number of questions:</div>
                <div class="question-options">
                    <div class="question-option selected" data-questions="10">10 Questions</div>
                    <div class="question-option" data-questions="20">20 Questions</div>
                    <div class="question-option" data-questions="30">30 Questions</div>
                </div>
            </div>
            <button class="start-btn" id="startBtn">Start Quiz</button>
        </div>

        <!-- Quiz Screen -->
        <div class="quiz-screen" id="quizScreen">
            <h1>📝 AI Text Detector Quiz</h1>

            <div class="progress-indicator" id="progressIndicator">
                Question <span id="currentQuestion">0</span> of <span id="totalQuestions">0</span>
            </div>

            <div class="score-board">
                <div class="score-item">
                    <div class="score-label">Score</div>
                    <div class="score-value" id="score">0/0</div>
                </div>
                <div class="score-item">
                    <div class="score-label">Correct</div>
                    <div class="score-value" id="correct">0</div>
                </div>
                <div class="score-item">
                    <div class="score-label">Accuracy</div>
                    <div class="score-value" id="accuracy">0%</div>
                </div>
            </div>

            <div class="quiz-text-container">
                <div class="text-loading" id="textLoading">
                    <div class="spinner"></div>
                </div>
                <div id="quizText" class="quiz-text" style="display: none;"></div>
            </div>

            <div class="answer-buttons" id="answerButtons">
                <button class="btn btn-ai" id="aiBtn" disabled>🤖 AI-generated</button>
                <button class="btn btn-human" id="humanBtn" disabled>👤 Human-written</button>
            </div>

            <div class="loading" id="loading">
                <div class="spinner"></div>
                <p>Checking answer...</p>
            </div>

            <div class="result-panel" id="resultPanel">
                <div class="result-title" id="resultTitle"></div>
                <div class="result-details" id="resultDetails"></div>
                <div class="model-prediction" id="modelPrediction"></div>
                <button class="btn btn-next" id="nextBtn">Next Question →</button>
            </div>

            <div id="error" class="error" style="display: none;"></div>
        </div>

        <!-- Completion Screen -->
        <div class="completion-screen" id="completionScreen">
            <h1 class="completion-title">🎉 Quiz Complete!</h1>
            <div class="completion-stats">
                <div class="completion-stat">
                    <div>Final Score</div>
                    <div class="completion-stat-value" id="finalScore">0/0</div>
                </div>
                <div class="completion-stat">
                    <div>Correct Answers</div>
                    <div class="completion-stat-value" id="finalCorrect">0</div>
                </div>
                <div class="completion-stat">
                    <div>Accuracy</div>
                    <div class="completion-stat-value" id="finalAccuracy">0%</div>
                </div>
            </div>
            <button class="start-btn" id="restartBtn">Play Again</button>
        </div>
    </div>

    <script>
        // State
        let currentTextId = null;
        let currentTrueLabel = null;
        let score = { correct: 0, total: 0 };
        let answerSelected = false;
        let totalQuestions = 10;
        let currentQuestion = 0;
        let usedTextIds = new Set(); // Track used texts to avoid repeats

        // Elements
        const startScreen = document.getElementById('startScreen');
        const quizScreen = document.getElementById('quizScreen');
        const completionScreen = document.getElementById('completionScreen');
        const startBtn = document.getElementById('startBtn');
        const restartBtn = document.getElementById('restartBtn');
        const quizText = document.getElementById('quizText');
        const textLoading = document.getElementById('textLoading');
        const aiBtn = document.getElementById('aiBtn');
        const humanBtn = document.getElementById('humanBtn');
        const loading = document.getElementById('loading');
        const resultPanel = document.getElementById('resultPanel');
        const resultTitle = document.getElementById('resultTitle');
        const resultDetails = document.getElementById('resultDetails');
        const modelPrediction = document.getElementById('modelPrediction');
        const nextBtn = document.getElementById('nextBtn');
        const error = document.getElementById('error');
        const scoreEl = document.getElementById('score');
        const correctEl = document.getElementById('correct');
        const accuracyEl = document.getElementById('accuracy');
        const progressIndicator = document.getElementById('progressIndicator');
        const currentQuestionEl = document.getElementById('currentQuestion');
        const totalQuestionsEl = document.getElementById('totalQuestions');
        const finalScoreEl = document.getElementById('finalScore');
        const finalCorrectEl = document.getElementById('finalCorrect');
        const finalAccuracyEl = document.getElementById('finalAccuracy');

        // Question selector
        const questionOptions = document.querySelectorAll('.question-option');
        questionOptions.forEach(option => {
            option.addEventListener('click', () => {
                questionOptions.forEach(opt => opt.classList.remove('selected'));
                option.classList.add('selected');
                totalQuestions = parseInt(option.dataset.questions);
            });
        });

        // Start quiz
        startBtn.addEventListener('click', () => {
            // Reset state
            score = { correct: 0, total: 0 };
            currentQuestion = 0;
            usedTextIds.clear();
            updateProgress();
            updateScore();

            startScreen.style.display = 'none';
            completionScreen.classList.remove('active');
            quizScreen.classList.add('active');
            loadNextQuestion();
        });

        // Restart quiz
        restartBtn.addEventListener('click', () => {
            completionScreen.classList.remove('active');
            startScreen.style.display = 'block';
        });

        // Answer buttons
        aiBtn.addEventListener('click', () => checkAnswer('AI-generated'));
        humanBtn.addEventListener('click', () => checkAnswer('Human-written'));
        nextBtn.addEventListener('click', loadNextQuestion);

        async function loadNextQuestion() {
            // Check if quiz is complete (after answering all questions)
            if (currentQuestion >= totalQuestions) {
                showCompletionScreen();
                return;
            }

            // Reset state
            answerSelected = false;
            resultPanel.classList.remove('show');
            aiBtn.disabled = false;
            humanBtn.disabled = false;
            quizText.style.display = 'none';
            textLoading.style.display = 'flex';
            hideError();

            try {
                // Get random text (try up to 10 times to avoid repeats)
                let attempts = 0;
                let data;
                do {
                    const response = await fetch('/quiz/text');
                    if (!response.ok) {
                        throw new Error('Failed to load quiz text');
                    }
                    data = await response.json();
                    attempts++;
                } while (usedTextIds.has(data.text_id) && attempts < 10);

                currentTextId = data.text_id;
                currentTrueLabel = data.true_label;
                usedTextIds.add(currentTextId);

                // Display text
                updateProgress(); // Update progress when loading new question
                quizText.textContent = data.text;
                textLoading.style.display = 'none';
                quizText.style.display = 'block';
            } catch (err) {
                showError(err.message);
                textLoading.style.display = 'none';
            }
        }

        async function checkAnswer(userAnswer) {
            if (answerSelected || !currentTextId) return;

            answerSelected = true;
            aiBtn.disabled = true;
            humanBtn.disabled = true;
            showLoading();
            hideError();

            try {
                const response = await fetch('/quiz/check', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json'
                    },
                    body: JSON.stringify({
                        text_id: currentTextId,
                        answer: userAnswer
                    })
                });

                if (!response.ok) {
                    throw new Error('Failed to check answer');
                }

                const data = await response.json();

                // Update score
                score.total++;
                currentQuestion++;
                if (data.is_correct) {
                    score.correct++;
                }
                updateScore();
                updateProgress();

                // Show result
                showResult(data);

                // Update button text if quiz is complete
                if (currentQuestion >= totalQuestions) {
                    nextBtn.textContent = 'View Results →';
                } else {
                    nextBtn.textContent = 'Next Question →';
                }
            } catch (err) {
                showError(err.message);
            } finally {
                hideLoading();
            }
        }

        function showResult(data) {
            // Result title
            if (data.is_correct) {
                resultTitle.textContent = '✓ Correct!';
                resultTitle.className = 'result-title result-correct';
            } else {
                resultTitle.textContent = '✗ Incorrect';
                resultTitle.className = 'result-title result-incorrect';
            }

            // Result details
            resultDetails.innerHTML = `
                <div class="result-detail-item">
                    <strong>Your Answer:</strong> ${data.user_answer}
                </div>
                <div class="result-detail-item">
                    <strong>Correct Answer:</strong> ${data.true_label}
                </div>
            `;

            // Model prediction
            modelPrediction.innerHTML = `
                <div class="model-prediction-title">🤖 AI Model Prediction:</div>
                <div class="result-detail-item">
                    <strong>Prediction:</strong> ${data.model_prediction}
                </div>
                <div class="result-detail-item">
                    <strong>Confidence:</strong> ${(data.model_confidence * 100).toFixed(1)}%
                </div>
                <div class="probabilities">
                    ${Object.entries(data.model_probabilities).map(([label, prob]) => `
                        <div class="prob-item">
                            <div class="prob-label">${label}</div>
                            <div class="prob-bar">
                                <div class="prob-fill" style="width: ${prob * 100}%"></div>
                            </div>
                            <div class="prob-value">${(prob * 100).toFixed(1)}%</div>
                        </div>
                    `).join('')}
                </div>
            `;

            resultPanel.classList.add('show');
        }

        function updateScore() {
            scoreEl.textContent = `${score.correct}/${score.total}`;
            correctEl.textContent = score.correct;
            const accuracy = score.total > 0 ? (score.correct / score.total * 100).toFixed(0) : 0;
            accuracyEl.textContent = `${accuracy}%`;
        }

        function updateProgress() {
            // Show current question number (1-indexed for display)
            currentQuestionEl.textContent = currentQuestion + 1;
            totalQuestionsEl.textContent = totalQuestions;
        }

        function showCompletionScreen() {
            quizScreen.classList.remove('active');
            completionScreen.classList.add('active');

            finalScoreEl.textContent = `${score.correct}/${score.total}`;
            finalCorrectEl.textContent = score.correct;
            const accuracy = score.total > 0 ? (score.correct / score.total * 100).toFixed(0) : 0;
            finalAccuracyEl.textContent = `${accuracy}%`;
        }

        function showLoading() {
            loading.style.display = 'block';
        }

        function hideLoading() {
            loading.style.display = 'none';
        }

        function showError(message) {
            error.textContent = message;
            error.style.display = 'block';
        }

        function hideError() {
            error.style.display = 'none';
        }
    </script>
</body>
</html>

'''

with open('templates/index.html', 'w') as f:
    f.write(html_template)

print('✓ Created templates/index.html')

✓ Created templates/index.html


In [8]:
os.makedirs('ai_text_detector', exist_ok=True)
with open('ai_text_detector/__init__.py', 'w') as f:
    f.write('# AI Text Detector Package\n')

models_code = '''import os
import sys

# Disable tokenizer parallelism and MPS on macOS
if os.getenv("TOKENIZERS_PARALLELISM") is None:
    os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
import torch.nn as nn
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, AutoModel, PreTrainedModel

class DesklibAIDetectionModel(PreTrainedModel):
    """Desklib AI Detection Model - Pre-trained model for AI text detection"""
    config_class = AutoConfig

    def __init__(self, config):
        super().__init__(config)
        # Initialize the base transformer model
        self.model = AutoModel.from_config(config)
        # Define a classifier head
        self.classifier = nn.Linear(config.hidden_size, 1)
        # Initialize weights
        self.init_weights()

    def forward(self, input_ids, attention_mask=None, labels=None):
        # Forward pass through the transformer
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs[0]

        # Mean pooling
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, dim=1)
        sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)
        pooled_output = sum_embeddings / sum_mask

        # Classifier
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits.view(-1), labels.float())

        output = {"logits": logits}
        if loss is not None:
            output["loss"] = loss
        return output

class DetectorModel:
    def __init__(self, model_name="desklib/ai-text-detector-v1.01", use_desklib=True):
        """
        Initialize detector model.

        Args:
            model_name: Model name or path. Defaults to Desklib pre-trained model.
            use_desklib: If True, use Desklib model architecture. If False, use standard classification.
        """
        self.model_name = model_name
        self.use_desklib = use_desklib

        if use_desklib and "desklib" in model_name:
            # Try to load Desklib model, but fallback if MPS issues occur
            if sys.platform == "darwin":
                # On macOS: try multiple loading strategies
                try:
                    # Strategy 1: Load with low_cpu_mem_usage and explicit CPU
                    print("Attempting to load Desklib model...")
                    self.tokenizer = AutoTokenizer.from_pretrained(model_name)
                    config = AutoConfig.from_pretrained(model_name)

                    # Try loading with safetensors if available
                    try:
                        from transformers import AutoModel
                        # Load base model first
                        base_model = AutoModel.from_pretrained(
                            model_name,
                            torch_dtype=torch.float32,
                            low_cpu_mem_usage=True,
                            device_map="cpu"
                        )
                        # Create Desklib model wrapper
                        self.model = DesklibAIDetectionModel(config)
                        self.model.model = base_model
                        self.model = self.model.to("cpu")
                        # Load classifier weights
                        from transformers.utils import cached_file
                        try:
                            classifier_path = cached_file(model_name, "pytorch_model.bin")
                            state_dict = torch.load(classifier_path, map_location="cpu")
                            # Only load classifier weights
                            classifier_dict = {k: v for k, v in state_dict.items() if "classifier" in k}
                            if classifier_dict:
                                self.model.load_state_dict(classifier_dict, strict=False)
                        except:
                            pass  # Use initialized classifier
                        self.model.eval()
                        print("✅ Desklib model loaded successfully!")
                    except Exception as e:
                        print(f"⚠️  Desklib model loading failed: {e}")
                        print("Falling back to DistilBERT model...")
                        raise
                except:
                    # Fallback to a smaller, simpler model
                    print("Using DistilBERT as fallback (smaller, more compatible)")
                    self.use_desklib = False
                    self.model = AutoModelForSequenceClassification.from_pretrained(
                        "distilbert-base-uncased",
                        num_labels=2
                    )
                    self.tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
                    self.model = self.model.to("cpu")
            else:
                # Non-macOS (Colab/Linux): standard loading
                try:
                    print(f"Loading Desklib model from {model_name}...")
                    self.tokenizer = AutoTokenizer.from_pretrained(model_name)
                    config = AutoConfig.from_pretrained(model_name)

                    # Try loading with from_pretrained first
                    try:
                        self.model = DesklibAIDetectionModel.from_pretrained(
                            model_name,
                            torch_dtype=torch.float32,
                            low_cpu_mem_usage=True
                        )
                        print("✅ Desklib model loaded via from_pretrained!")
                    except Exception as load_err:
                        # If from_pretrained fails, manually load weights
                        print(f"⚠️  from_pretrained failed: {load_err}")
                        print("Trying manual weight loading...")

                        # Create model instance
                        self.model = DesklibAIDetectionModel(config)

                        # Load weights manually
                        from transformers.utils import cached_file
                        from safetensors.torch import load_file as safetensors_load

                        # Try safetensors first
                        try:
                            safetensors_path = cached_file(model_name, "model.safetensors")
                            state_dict = safetensors_load(safetensors_path)
                            self.model.load_state_dict(state_dict, strict=False)
                            print("✅ Loaded weights from safetensors!")
                        except:
                            # Fallback to pytorch_model.bin
                            try:
                                bin_path = cached_file(model_name, "pytorch_model.bin")
                                state_dict = torch.load(bin_path, map_location="cpu")
                                self.model.load_state_dict(state_dict, strict=False)
                                print("✅ Loaded weights from pytorch_model.bin!")
                            except Exception as bin_err:
                                raise Exception(f"Failed to load weights: {bin_err}")

                    # Ensure model is on CPU
                    self.model = self.model.to("cpu")
                    self.model.eval()
                    print("✅ Desklib model loaded successfully!")
                except Exception as e:
                    print(f"⚠️  Failed to load Desklib model: {e}")
                    print("Falling back to RoBERTa base model...")
                    import traceback
                    traceback.print_exc()
                    # Fallback to RoBERTa
                    self.use_desklib = False
                    self.model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
                    self.tokenizer = AutoTokenizer.from_pretrained("roberta-base", use_fast=True)
                    self.model = self.model.to("cpu")
        else:
            # Fallback to standard classification model (when use_desklib=False)
            print(f"Loading standard model: {model_name}")
            self.model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
            self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
            self.use_desklib = False

    def predict(self, text, max_length=768, threshold=0.5):
        """
        Predict if text is AI-generated.

        Args:
            text: Input text to classify
            max_length: Maximum sequence length
            threshold: Probability threshold for classification

        Returns:
            tuple: (probability, label) where label is 1 for AI-generated, 0 for human
        """
        # Tokenize
        encoded = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_tensors='pt'
        )

        input_ids = encoded['input_ids']
        attention_mask = encoded['attention_mask']

        # Get device
        device = next(self.model.parameters()).device
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        # Predict
        self.model.eval()
        with torch.no_grad():
            if self.use_desklib:
                outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs["logits"]
                probability = torch.sigmoid(logits).item()
            else:
                outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
                probs = torch.softmax(outputs.logits, dim=1)
                # For standard models: prob[0] = human, prob[1] = AI
                probability = probs[0][1].item()

            label = 1 if probability >= threshold else 0

        return probability, label

    def save(self, path: str):
        self.model.save_pretrained(path)
        self.tokenizer.save_pretrained(path)

    @classmethod
    def load(cls, path: str):
        # Try to detect if it's a Desklib model
        try:
            config = AutoConfig.from_pretrained(path)
            # Check if it has the Desklib architecture
            if hasattr(config, 'model_type') and 'deberta' in config.model_type.lower():
                model = DesklibAIDetectionModel.from_pretrained(path)
                tokenizer = AutoTokenizer.from_pretrained(path)
                obj = cls.__new__(cls)
                obj.model_name = path
                obj.model = model
                obj.tokenizer = tokenizer
                obj.use_desklib = True
                return obj
        except:
            pass

        # Fallback to standard model
        model = AutoModelForSequenceClassification.from_pretrained(path)
        tokenizer = AutoTokenizer.from_pretrained(path, use_fast=True)
        obj = cls.__new__(cls)
        obj.model_name = path
        obj.model = model
        obj.tokenizer = tokenizer
        obj.use_desklib = False
        return obj
'''
# Remove MPS-specific code for Colab\n
models_code_colab = models_code.replace('if sys.platform == \"darwin\":', 'if False:')  # Disable MPS checks\n

with open('ai_text_detector/models.py', 'w') as f:
    f.write(models_code_colab)
print('✓ Created ai_text_detector/models.py')

✓ Created ai_text_detector/models.py


## Step 4: Deploy to Hugging Face Spaces 🚀

Choose one of the following options:

### Option A: Deploy to Hugging Face Spaces (Permanent URL)
This creates a permanent, shareable URL that never expires!

In [15]:
# Create Gradio quiz app for HF Spaces
gradio_code = """"""
"""
import os
import sys
import random
import logging

# No MPS code needed for HF Spaces (runs on Linux)
import torch
import gradio as gr
import pandas as pd

from ai_text_detector.models import DetectorModel
from src.quiz_dataset_loader import QuizDatasetLoader

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Global variables
detector = None
quiz_loader = None
current_question = 0
score = {"correct": 0, "total": 0}
current_sample = None
used_ids = set()

def load_model():
    global detector

    model_path = "models/ai_detector"

    # Check if model directory exists AND has model files
    has_model = False
    if os.path.exists(model_path):
        required_files = ["config.json", "pytorch_model.bin"]
        has_model = all(os.path.exists(os.path.join(model_path, f)) for f in required_files)

    if has_model:
        try:
            logger.info(f"Loading trained model from {model_path}")
            detector = DetectorModel.load(model_path)
        except Exception as e:
            logger.error(f"Failed to load model: {e}")
            logger.info("Using Desklib pre-trained model instead.")
            detector = DetectorModel("desklib/ai-text-detector-v1.01", use_desklib=True)
    else:
        logger.info("No trained model found. Loading Desklib pre-trained AI detector model...")
        try:
            detector = DetectorModel("desklib/ai-text-detector-v1.01", use_desklib=True)
            logger.info("✅ Desklib model loaded successfully!")
        except Exception as e:
            logger.error(f"⚠️  Failed to load Desklib model: {e}")
            logger.info("Falling back to RoBERTa base model...")
            detector = DetectorModel("roberta-base", use_desklib=False)

    return detector

def load_quiz_dataset(data_dir='data'):
    global quiz_loader

    logger.info(f"Loading quiz dataset from {data_dir}...")

    try:
        quiz_loader = QuizDatasetLoader(data_dir=data_dir)

        if len(quiz_loader) > 0:
            msg = f"✓ Quiz dataset loaded: {len(quiz_loader)} text samples"
            print(msg)
            logger.info(msg)
            return quiz_loader
        else:
            logger.warning("QuizDatasetLoader created but has 0 samples")
    except Exception as e:
        error_msg = f"Warning: Could not load quiz dataset: {e}"
        print(error_msg)
        logger.error(error_msg, exc_info=True)
        quiz_loader = None

    return None

# Initialize on import
load_model()
load_quiz_dataset()

def start_quiz(num_questions):
    global current_question, score, used_ids, current_sample

    current_question = 0
    score = {"correct": 0, "total": 0}
    used_ids.clear()
    current_sample = None

    return get_next_question(num_questions) + (gr.update(visible=True), gr.update(visible=False))

def get_next_question(num_questions):
    global current_sample, quiz_loader

    if quiz_loader is None or len(quiz_loader) == 0:
        return (
            "⚠️ No quiz dataset available. Please ensure CSV files exist in the data/ directory.",
            gr.update(visible=False),
            gr.update(visible=False),
            f"Question 0/{num_questions}",
            "Score: 0/0 | Accuracy: 0%"
        )

    # Get random sample (avoid repeats)
    attempts = 0
    sample = None
    sample_id = None

    while attempts < 10:
        sample_id, sample = quiz_loader.get_random_sample()
        if sample_id not in used_ids:
            break
        attempts += 1

    if sample is None:
        return (
            "⚠️ Could not load quiz sample. Please try again.",
            gr.update(visible=False),
            gr.update(visible=False),
            f"Question 0/{num_questions}",
            "Score: 0/0 | Accuracy: 0%"
        )

    used_ids.add(sample_id)
    current_sample = sample

    progress = f"Question {current_question + 1}/{num_questions}"
    score_text = f"Score: {score['correct']}/{score['total']} | Accuracy: {(score['correct']/score['total']*100) if score['total'] > 0 else 0:.0f}%"

    return (
        sample['text'],
        gr.update(visible=True),
        gr.update(visible=True),
        progress,
        score_text
    )

def check_answer(user_answer, num_questions):
    global current_question, score, current_sample, detector

    if current_sample is None:
        return (
            "⚠️ No question loaded. Please start a new quiz.",
            gr.update(visible=False),
            gr.update(visible=False),
            "",
            "",
            f"Question 0/{num_questions}",
            "Score: 0/0 | Accuracy: 0%"
        )

    text = current_sample['text']
    true_label = current_sample['label_name']

    # Get model prediction
    try:
        ai_prob, predicted_label = detector.predict(text, max_length=768, threshold=0.5)
        predicted_label_name = 'AI-generated' if predicted_label == 1 else 'Human-written'
        human_prob = 1 - ai_prob

        # Check if user is correct
        is_correct = user_answer == true_label

        # Update score
        score['total'] += 1
        current_question += 1
        if is_correct:
            score['correct'] += 1

        # Create result message
        result_icon = "✅" if is_correct else "❌"
        result_text = f"{result_icon} {'Correct!' if is_correct else 'Incorrect'}\n\n"
        result_text += f"**Your Answer:** {user_answer}\n"
        result_text += f"**Correct Answer:** {true_label}\n\n"
        result_text += f"**🤖 AI Model Prediction:** {predicted_label_name}\n"
        result_text += f"**Confidence:** {(ai_prob if predicted_label == 1 else human_prob) * 100:.1f}%\n\n"
        result_text += f"**Probabilities:**\n"
        result_text += f"- AI-generated: {ai_prob * 100:.1f}%\n"
        result_text += f"- Human-written: {human_prob * 100:.1f}%"

        progress = f"Question {current_question}/{num_questions}"
        accuracy = (score['correct'] / score['total'] * 100) if score['total'] > 0 else 0
        score_text = f"Score: {score['correct']}/{score['total']} | Accuracy: {accuracy:.0f}%"

        # Check if quiz is complete
        if current_question >= num_questions:
            final_text = result_text + f"\n\n🎉 **Quiz Complete!**\n\n"
            final_text += f"**Final Score:** {score['correct']}/{score['total']}\n"
            final_text += f"**Final Accuracy:** {accuracy:.0f}%"
            return (
                final_text,
                gr.update(visible=False),
                gr.update(visible=False),
                gr.update(visible=True, value="🎉 Quiz Complete! Play Again?"),
                "",
                progress,
                score_text
            )

        return (
            result_text,
            gr.update(visible=False),
            gr.update(visible=True),
            "",
            "",
            progress,
            score_text
        )

    except Exception as e:
        logger.error(f"Error checking answer: {e}")
        return (
            f"⚠️ Error processing answer: {str(e)}",
            gr.update(visible=False),
            gr.update(visible=False),
            "",
            "",
            f"Question {current_question}/{num_questions}",
            f"Score: {score['correct']}/{score['total']} | Accuracy: {(score['correct']/score['total']*100) if score['total'] > 0 else 0:.0f}%"
        )

def next_question(num_questions):
    if current_question >= num_questions:
        return start_quiz(num_questions)
    return get_next_question(num_questions) + (gr.update(visible=True), gr.update(visible=False))

# Create Gradio interface
with gr.Blocks(title="AI Text Detector Quiz", theme=gr.themes.Soft()) as app:
    gr.Markdown("# 📝 AI Text Detector Quiz")
    gr.Markdown("Test your ability to distinguish AI-generated text from human-written text!")

    with gr.Row():
        num_questions = gr.Dropdown(
            choices=[10, 20, 30],
            value=10,
            label="Number of Questions",
            interactive=True
        )
        start_btn = gr.Button("Start Quiz", variant="primary", size="lg")

    progress_display = gr.Markdown("", visible=False)
    score_display = gr.Markdown("", visible=False)

    with gr.Row(visible=False) as quiz_row:
        with gr.Column():
            text_display = gr.Textbox(
                label="Text Sample",
                lines=10,
                max_lines=15,
                interactive=False,
                show_label=True
            )

            with gr.Row():
                ai_btn = gr.Button("🤖 AI-generated", variant="stop", size="lg")
                human_btn = gr.Button("👤 Human-written", variant="primary", size="lg")

        with gr.Column():
            result_display = gr.Markdown("", visible=False)
            next_btn = gr.Button("Next Question →", visible=False, variant="secondary")
            restart_btn = gr.Button("🎉 Quiz Complete! Play Again?", visible=False, variant="primary")

    # Event handlers
    start_btn.click(
        fn=start_quiz,
        inputs=num_questions,
        outputs=[text_display, quiz_row, progress_display, progress_display, score_display]
    )

    ai_btn.click(
        fn=check_answer,
        inputs=[gr.State("AI-generated"), num_questions],
        outputs=[result_display, ai_btn, human_btn, restart_btn, next_btn, progress_display, score_display]
    )

    human_btn.click(
        fn=check_answer,
        inputs=[gr.State("Human-written"), num_questions],
        outputs=[result_display, ai_btn, human_btn, restart_btn, next_btn, progress_display, score_display]
    )

    next_btn.click(
        fn=next_question,
        inputs=num_questions,
        outputs=[text_display, ai_btn, human_btn, progress_display, score_display, restart_btn, next_btn]
    )

    restart_btn.click(
        fn=start_quiz,
        inputs=num_questions,
        outputs=[text_display, quiz_row, progress_display, progress_display, score_display]
    )

if __name__ == "__main__":
    app.launch(share=True)

"""

with open('gradio_quiz_app.py', 'w') as f:
    f.write(gradio_code)

print('✓ Created gradio_quiz_app.py')

✓ Created gradio_quiz_app.py


In [16]:
# Create README.md for Hugging Face Space
space_readme = """---
title: AI Text Detector Quiz
emoji: 📝
colorFrom: blue
colorTo: purple
sdk: gradio
app_file: gradio_quiz_app.py
pinned: false
---

# AI Text Detector Quiz

A fun interactive quiz game where you test your ability to distinguish AI-generated text from human-written text!

## How to Play

1. **Choose** the number of questions (10, 20, or 30)
2. **Read** each text sample carefully
3. **Guess** whether it's AI-generated or Human-written
4. **See** if you're correct and compare with the AI model's prediction
5. **Track** your score and accuracy throughout the quiz

## Features

- 🎮 Interactive quiz interface
- 📊 Real-time scoring and accuracy tracking
- 🤖 See AI model predictions after each answer
- 📈 View confidence scores and probabilities
- 🎯 Multiple difficulty levels (10/20/30 questions)

## Dataset

The quiz uses text samples from the dataset in the `data/` folder. Make sure your CSV files have:
- A `text` column with the text samples
- A `label` column with values "AI-generated" or "Human-written"

## Model

Uses the Desklib AI Text Detector model for predictions. The model automatically loads on startup.

"""

with open('README.md', 'w') as f:
    f.write(space_readme)

print('✓ Created README.md for Hugging Face Space')

✓ Created README.md for Hugging Face Space


### Deploy to Hugging Face Spaces

**Prerequisites:**
1. Create a Hugging Face account at [huggingface.co/join](https://huggingface.co/join)
2. Get your access token from [huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)
   - Click "New token"
   - Name it (e.g., "colab-deploy")
   - Select "Write" permissions
   - Copy the token

In [17]:
# Install Hugging Face Hub and Gradio
%pip install -q gradio huggingface_hub

In [18]:
# Login to Hugging Face
from huggingface_hub import login

# Paste your token when prompted
login()

print('✅ Logged in to Hugging Face!')

✅ Logged in to Hugging Face!


In [19]:
# Deploy to Hugging Face Spaces!
!gradio deploy

print('\n🚀 Your quiz app is being deployed to Hugging Face Spaces!')
print('\n⏳ This usually takes 5-10 minutes. Check your Spaces dashboard:')
print('   https://huggingface.co/spaces')

Need 'write' access token to create a Spaces repo.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

Enter your token (input will not be visible): ^C

🚀 Your quiz app is being deployed to Hugging Face Spaces!

⏳ This usually takes 5-10 minutes. Check your Spaces dashboard:
   https://huggingface.co/spaces
